Memoria TFG. Tema 3 - Ejemplo de ejecución en el cluster.

Ejemplo extraído del manual de MRJob de Steve Johnson que nos muestra la palabra mas usada en un texto determinado. Lo hace en dos pasos. En el primer paso cuenta las palabras, y en el segundo paso extrae de la cuenta que ha hecho en el primer paso, la palabra con mayor número de apariciones.

In [21]:
%%writefile palabramasusada.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep

import re

WORD_RE = re.compile(r"[\w']+")

class MRMostUsedWord(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.mapper_extrae_palabras,
                combiner=self.combiner_cuenta_palabras,
                reducer=self.reducer_cuenta_palabras),
            MRStep(reducer=self.reducer_encuentra_palabra_mas_usada)
        ]
    
    def mapper_extrae_palabras(self, _, line):
        # producimos cada palabra en la linea
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)
    
    def combiner_cuenta_palabras(self, word, counts):
        # Sumamos las palabras vistas hasta ahora
        yield (word, sum(counts))
    
    def reducer_cuenta_palabras(self, word, counts):
        # Enviamos todo al siguiente reducer
        # para que encuentre la palabra más usada
        yield None, (sum(counts), word)
        
        # Descartamos la clave, nos hace falta la palabra y el número
    def reducer_encuentra_palabra_mas_usada(self, _, word_count_pairs):
        yield max(word_count_pairs)
        
if __name__ == '__main__':
    MRMostUsedWord.run()

Overwriting palabramasusada.py


In [22]:
!python palabramasusada.py archivos_datos/quijote.txt

20628	"que"


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\palabramasusada.manuel.20201126.184903.433642
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\cyber\AppData\Local\Temp\palabramasusada.manuel.20201126.184903.433642\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\palabramasusada.manuel.20201126.184903.433642\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\palabramasusada.manuel.20201126.184903.433642...


La palabra más usada en el Quijote es "que" con 20628 apariciones.